In [1]:
data_var = '2024-02-19'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
4834,2024-02-19,Argentina Liga A,21:00,Instituto de Cordoba,Platense,1.25,3.49,155.5,1.85,1.80,-9.5,1.97,1.70,GSO12n51,0.800000,0.286533,0.540541,0.555556,0.086533,0.0,0.4,NaN,NaN,104.870,26.466802,0.252377,174.718,118.671901,0.679220,89.428,213.558,87.32,378.84,0.0,0.0,0.0,0.0,0.668320,0.019373,0.104043,-0.18,-0.036,-6.944444,0.792999,0.8,0.007001,-0.01,-0.002,-1245.000000,0.000000,0.0,0.000000
4835,2024-02-19,China Wcba Feminina,08:30,Inner Mongolia F,Shandong F,1.02,10.50,156.5,1.82,1.82,-23.5,1.92,1.72,OKIQuBj6,0.980392,0.095238,0.549451,0.549451,0.075630,0.8,0.6,NaN,NaN,114.006,28.471334,0.249735,190.800,113.706345,0.595945,112.184,183.716,164.01,266.66,0.0,0.0,0.0,0.0,1.163780,0.000000,0.077704,-0.46,-0.092,-0.217391,0.000000,0.0,0.000000,-2.87,-0.574,-16.550523,0.000000,0.0,0.000000
4836,2024-02-19,China Wcba Feminina,08:30,Shaanxi Tianze F,Liaoning F,2.19,1.59,147.5,1.86,1.75,1.5,2.05,1.65,2BHUvV5C,0.456621,0.628931,0.537634,0.571429,0.085552,0.4,0.6,NaN,NaN,265.388,256.172378,0.965275,519.516,279.719538,0.538423,409.298,516.974,206.96,430.92,0.0,0.0,0.0,0.0,0.224478,0.043092,0.152888,-5.00,-1.000,-1.190000,0.000000,0.0,0.000000,-5.00,-1.000,-0.590000,0.000000,0.0,0.000000
4837,2024-02-19,China Wcba Feminina,08:30,Wuhan F,Xinjiang F,8.05,1.05,138.5,1.83,1.82,14.5,1.97,1.70,pz7ZwkLI,0.124224,0.952381,0.546448,0.549451,0.076605,0.4,0.6,NaN,NaN,209.996,230.623123,1.098226,276.458,214.821830,0.777051,206.086,183.350,97.50,95.16,0.0,0.0,0.0,0.0,1.087857,0.003875,0.104043,-0.65,-0.130,-54.230769,0.576586,0.6,0.023414,2.98,0.596,0.083893,0.000000,0.0,0.000000
4838,2024-02-19,Coreia Do Sul Wkbl Feminina,07:00,Woori WON F,KB Stars F,2.27,1.57,132.5,1.85,1.76,1.5,2.07,1.63,lYr4XrhT,0.440529,0.636943,0.540541,0.568182,0.077471,0.6,0.6,NaN,NaN,91.338,18.208954,0.199358,85.552,13.551899,0.158405,93.746,90.330,80.52,104.55,0.0,0.0,0.0,0.0,0.257799,0.035257,0.168177,-1.41,-0.282,-4.503546,0.000000,0.0,0.000000,0.17,0.034,16.764706,0.826276,0.8,-0.026276
4839,2024-02-19,Eua Ncaa,20:00,Lafayette,Colgate,4.69,1.19,131.5,1.84,1.92,7.5,2.07,1.65,zDwXca3b,0.213220,0.840336,0.543478,0.520833,0.053556,0.8,0.6,NaN,NaN,95.096,23.163723,0.243583,88.344,12.153349,0.137568,104.510,101.066,77.44,89.68,0.0,0.0,0.0,0.0,0.841794,0.030090,0.159669,-1.58,-0.316,-11.677215,0.000000,0.0,0.000000,1.89,0.378,0.502646,0.511949,0.7,0.188051
4840,2024-02-19,Eua Ncaa,21:00,SE Louisiana,Lamar,1.84,1.92,144.5,1.88,1.88,-2.5,2.02,1.69,QsI7e2hU,0.543478,0.520833,0.531915,0.531915,0.064312,0.6,0.6,NaN,NaN,175.064,136.643219,0.780533,127.332,29.976460,0.235420,153.662,205.598,87.48,134.42,0.0,0.0,0.0,0.0,0.030090,0.000000,0.125793,4.06,0.812,1.034483,0.000000,0.0,0.000000,-3.57,-0.714,-1.288515,0.424439,0.3,-0.124439
4841,2024-02-19,Eua Ncaa,21:00,Virginia Tech,Virginia,1.71,2.14,126.5,1.88,1.88,-3.5,1.97,1.69,8dI5L1Oa,0.584795,0.467290,0.531915,0.531915,0.052085,0.4,1.0,NaN,NaN,202.442,143.791492,0.710285,116.122,40.717122,0.350641,126.976,126.636,122.01,144.00,0.0,0.0,0.0,0.0,0.157951,0.000000,0.108191,-0.78,-0.156,-4.551282,0.646954,0.7,0.053046,1.24,0.248,4.596774,0.000000,0.0,0.000000
4842,2024-02-19,Eua Ncaa,21:30,Alabama A&M,Arkansas-Pine Bluff,1.65,2.24,154.5,1.84,1.88,-4.5,2.02,1.69,WUF1KDEm,0.606061,0.446429,0.543478,0.531915,0.052489,0.2,0.4,NaN,NaN,198.062,113.274257,0.571913,166.152,84.210445,0.506828,134.860,236.114,85.60,227.92,0.0,0.0,0.0,0.0,0.214495,0.015207,0.125793,-0.21,-0.

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Israel Wbl Feminina,Hapoel Jerusalem F,Hapoel Petah Tikva F,148.5,1.88,0.5533,Over
